In [1]:
import sys
import pandas as pd
sys.path.append("..")

import chemspace as cs
from chemspace.Dataset.dataset_builder import DatasetBuilder

In [2]:
#DB = DatasetBuilder(compound_file_path='../chemspace/Dataset/Data/PubChem_compound_list_records.json.gz')
DB = DatasetBuilder(compound_file_path='../chemspace/Dataset/Data/CompoundDataset.csv')

In [4]:
DB.dataset
DB.dataset.to_csv('../chemspace/Dataset/Data/CompoundDataset.csv', index = True)